In [1]:
import base64
import json
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, Model, CodeConfiguration, OnlineRequestSettings, Environment
#from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, Model, Environment, JobService, Data, CodeConfiguration, OnlineRequestSettings, AmlCompute

from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

ml_client = MLClient.from_config(credential=credential)

Found the config file in: ./config.json


In [30]:
# Deployment
online_endpoint_name = 'SRN-endpoint'
registered_model_name = 'custom-model'
deployment_name = 'blue'
inference_env_name = 'inference-env'

In [12]:
# Let's pick the latest version of the model
latest_model_version = max([int(m.version) for m in ml_client.models.list(name= registered_model_name)])

print(f'Latest version of {registered_model_name} found: {latest_model_version}')

Latest version of SRG found: 1


In [6]:
# model_name = 'sklearn-diabetes'
# model_local_path = "sklearn-diabetes/model"
# model = ml_client.models.create_or_update(
#         Model(name=model_name, path=model_local_path, type=AssetTypes.MLFLOW_MODEL)
# )

model = ml_client.models.get(name=registered_model_name, label='latest')


In [15]:
endpoint_name = online_endpoint_name

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="An online endpoint to generate predictions",
    auth_mode="key",
    tags={"foo": "bar"},
)

In [16]:
ml_client.begin_create_or_update(endpoint)

In [ ]:

scoring_environment = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.1-cudnn8-ubuntu20.04:latest",
    conda_file="./scoring-env.yaml",
    name=inference_env_name,
    description="Brain tumor segmentation inference environment")

ml_client.environments.create_or_update(scoring_environment)

In [32]:
latest_env_name = inference_env_name + ':latest' 
blue_deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=online_endpoint_name,
    model=model,
    environment=latest_env_name,
    code_configuration=CodeConfiguration(
        code="./",
        scoring_script="score.py"
    ),
    instance_type="Standard_NC4as_T4_v3",
    instance_count=1,
    request_settings= OnlineRequestSettings(request_timeout_ms = 90000),
)

In [33]:
deployment = ml_client.begin_create_or_update(blue_deployment)

Check: endpoint SRG-endpoint exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


...............................

In [34]:
endpoint = ml_client.online_endpoints.get(online_endpoint_name)
print(f"Endpoint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

Endpoint srg-endpoint provisioning state: Succeeded


In [ ]:
ml_client.online_deployments.get_logs(
    name=deployment_name, endpoint_name=online_endpoint_name, lines=50
)

In [35]:
# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'red': 0, 'blue': 0}
https://srg-endpoint.westeurope.inference.ml.azure.com/score


In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {deployment_name: 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()